In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val ACCOUNT_MATCHERS_CSV_FILE = "../../data/provision/account-matchers.csv"
val FUND_MATCHERS_CSV_FILE = "../../data/provision/fund-matchers.csv"
val EXCHANGE_MATCHERS_CSV_FILE = "../../data/provision/exchange-matchers.csv"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-6"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=9fb464c9-2db8-42b4-ae5f-2498c3b10931, username=Johann-6)

In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=9e8fcdb3-3e02-40f9-bbc0-bccaba3f908d, name=BT Economy RON, unit=Currency(value=RON))
AccountTO(id=5538c71c-931f-479b-b32b-5fe22ec46b5d, name=BT RON, unit=Currency(value=RON))
AccountTO(id=bb38fb97-61e3-4253-8916-566ff2d6f700, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=4379b9ff-79c7-4280-99b4-191bc7965683, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=cd5261a2-4873-435b-8d09-67b35abeb79a, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=c6e75c80-1e3b-4e58-b149-89308fa6c28b, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=d692f6ef-1e42-4dae-bc07-aa4f66ee7a3b, name=ING RON, unit=Currency(value=RON))
AccountTO(id=34ad79e6-3103-4d66-a499-0594d32b4fcb, name=Other, unit=Currency(value=RON))
AccountTO(id=4ab469be-2575-4841-9cbc-581882ec5daf, name=Revolut RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=e3bab84b-8b74-4bfa-80f3-1d1bd5d3d7e5, name=Expenses)
FundTO(id=2d9db7c9-90f3-4883-9cec-c0865f1cecec, name=Work Income)
FundTO(id=15b4d878-9a50-48ae-8e84-e39d2753ecc2, name=Savings)

In [5]:
import ro.jf.funds.importer.api.model.AccountMatcherTO

val accountMatchers = runBlocking {
    DataFrame.readCSV(ACCOUNT_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row -> AccountMatcherTO(row.getValue<String>("import_account_name"), AccountName(row.getValue<String>("account_name"))) }
}
accountMatchers.map { it.toString() }.joinToString(separator = "\n")

AccountMatcherTO(importAccountName=ING old, accountName=ING RON)
AccountMatcherTO(importAccountName=ING Economy old, accountName=ING Economy)
AccountMatcherTO(importAccountName=Cash RON, accountName=Cash RON)
AccountMatcherTO(importAccountName=Euro, accountName=Cash EUR)
AccountMatcherTO(importAccountName=Food Coupons, accountName=Food Coupons)
AccountMatcherTO(importAccountName=Revolut - Manual, accountName=Revolut RON)
AccountMatcherTO(importAccountName=Other, accountName=Other)
AccountMatcherTO(importAccountName=BT RON, accountName=BT RON)
AccountMatcherTO(importAccountName=BT Economy RON, accountName=BT Economy RON)

In [6]:
import ro.jf.funds.importer.api.model.*

val fundMatchers = runBlocking {
    DataFrame.readCSV(FUND_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_account" -> FundMatcherTO.ByAccount(
                    importAccountName = row.getValue<String>("import_account_name"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )
                
                "by_label" -> FundMatcherTO.ByLabel(
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label" -> FundMatcherTO.ByAccountLabel(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label_with_transfer" -> FundMatcherTO.ByAccountLabelWithTransfer(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    initialFundName = FundName(row.getValue<String>("initial_fund_name")),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                else -> error("fund matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByAccountLabelWithTransfer(importAccountName=ING Economy old, importLabel=Invest Profit Dist, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING old, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=BT RON, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=BT Economy RON, importLabel=Invest Profit Dist, initialFundName=Savings, fundName=Expenses)
ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabe

In [7]:
val exchangeMatchers = runBlocking {
    DataFrame.readCSV(EXCHANGE_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_label" -> ExchangeMatcherTO.ByLabel(
                    label = row.getValue<String>("label"),
                )

                else -> error("exchange matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByAccountLabelWithTransfer(importAccountName=ING Economy old, importLabel=Invest Profit Dist, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING old, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=BT RON, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=BT Economy RON, importLabel=Invest Profit Dist, initialFundName=Savings, fundName=Expenses)
ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabe

In [8]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

// TODO(Johann) load all files from the directory
val csvFiles = File("../../data/wallet/2019")
    .listFiles()?.toList() ?: error("no files found")
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    accountMatchers = accountMatchers,
    fundMatchers = fundMatchers,
    exchangeMatchers = exchangeMatchers,
)
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=c60957f1-b7bc-4041-9e9e-86299bf9d7c2, status=COMPLETED, reason=null)